# Dateien, Collections, bedingte Anweisungen, Schleifen, List Comprehensions
## Verwalte den Python-Kurs
Die Datei Lektionen_1_bis_3_Studierende.txt enthält Studierenden-Daten nach folgendem Schema: \<Name\>; \<Studiengang\>; \<Semester\>; \<Programmiererfahrung\>. Der Name ist eindeutig. Der Eintrag zur Programmiererfahrung kann verschiedene kommaseparierte Informationen enthalten, nach dem Schema: \<Erfahrung1\>, \<Erfahrung2\>, .... Pro Zeile sind die Daten einer Person erfasst. Du kannst die Datei in einem Texteditor öffnen, um dir das Format und die Inhalte anzuschauen.
## Aufgabe A
Lies die Daten aus der Datei ein und speichere sie strukturiert in einer Variablen `python_course`. Jede Person soll als Dictionary mit den in `keys` gegebenen Schlüsseln dargestellt werden. Die Programmiererfahrungen sollen pro Person in einem Set gespeichert werden. Der gesamte Kurs soll als Liste dargestellt werden.

Tipps: 
* Recherchiere [hier](https://docs.python.org/3/tutorial/inputoutput.html#methods-of-file-objects), wie du Dateien zeilenweise einlesen kannst. 
* Erinnere dich an die String-Funktion, mit der du einen String "zerschneiden" kannst.

In [2]:
#Die Schlüssel in dem Tupel (unveränderliche Liste) keys sollen zur Konstruktion der Dictionaries verwendet werden.
#Führe diese Zelle einmal zu Beginn aus, so dass der Interpreter den Namen kennt
keys = ("Name", "Studiengang", "Semester", "Programmiererfahrung")

In [3]:
python_course = [] #gesamter Kurs als Liste
filename = "Lektionen_1_bis_3_Studierende.txt"
#encoding="utf-8" sorgt dafür, dass auch die Umlaute korrekt dargestellt werden
#Anstelle des with-Statement können auch die Funktion open und die Methode close verwendet werden.
#Mit dem with-Statement wird die Datei automatisch geschlossen, siehe z.B. das Beispiel hier: https://docs.python.org/3/whatsnew/2.5.html#pep-343-the-with-statement
with open(filename, "r", encoding="utf-8") as file:
    for line in file: #eine andere Möglichkeit, die Dateiinhalte zeilenweise zu lesen, wäre die Verwendung der Methode readline
        items = line.strip().split("; ") #strip entfernt führende und finale Leerzeichen und Newlines, split teilt die Zeichenkette in line an jeder Sequenz Semikolon+Leerzeichen auf
        #items ist nun eine Liste der aufgeteilten Zeichenketten
        #Programmiererfahrungen sind durch Kommata separiert. Diese werden weiter aufgeteilt, um sie anschließend unten in einem Set zu speichern.
        programming_experiences = items[3].strip().split(", ")
        #Jede Person wird über ein Dictionary dargestellt
        #Programmiererfahrungen werden als Set innerhalb der Dictionaries dargestellt, damit effizient nach einer bestimmten Erfahrung abgefragt werden kann
        python_course.append({keys[0]:items[0], keys[1]:items[1], keys[2]:items[2], keys[3]:set(programming_experiences)}) 
print(python_course)

[{'Name': 'Robin', 'Studiengang': 'Künstliche Intelligenz', 'Semester': '3', 'Programmiererfahrung': {'Python'}}, {'Name': 'Christina', 'Studiengang': 'Cyber Security', 'Semester': '1', 'Programmiererfahrung': {'C/C++'}}, {'Name': 'Dominik', 'Studiengang': 'Robotik', 'Semester': '2', 'Programmiererfahrung': {'SPS'}}, {'Name': 'Yvonne', 'Studiengang': 'Elektrotechnik', 'Semester': '2', 'Programmiererfahrung': {'SPS', 'C/C++'}}, {'Name': 'Billel', 'Studiengang': 'Cyber Security', 'Semester': '1', 'Programmiererfahrung': {'keine'}}, {'Name': 'Paul', 'Studiengang': 'Cyber Security', 'Semester': '2', 'Programmiererfahrung': {'Fortran', 'C/C++'}}, {'Name': 'Christian', 'Studiengang': 'Elektrotechnik', 'Semester': '1', 'Programmiererfahrung': {'Java', 'SPS'}}, {'Name': 'Rahel', 'Studiengang': 'Cyber Security', 'Semester': '2', 'Programmiererfahrung': {'Java'}}, {'Name': 'Tomi', 'Studiengang': 'FinTech', 'Semester': '1', 'Programmiererfahrung': {'Python'}}, {'Name': 'Marie', 'Studiengang': 'Kü

## Aufgabe B
* Gib die Teilnehmenden zeilenweise aus, dass eine Person pro Zeile strukturiert (eine Person pro Zeile) aus
* Berechne das durchschnittliche Semester aller Teilnehmenden und gib dieses aus
* Gib die Anzahl der Personen aus, die bereits Erfahrung mit Python haben

In [4]:
sum_semester = 0
python_experience_counter = 0
for person in python_course: #Iteriere über alle Personen im Kurs
    print(person) #Gib die aktuelle Person aus
    #Summiere die Semester aller Personen, um später den Durchschnitt zu berechnen
    sum_semester += int(person[keys[2]])
    if "Python" in person[keys[3]]: #Zähle die Personen, die Python als Programmiererfahrung aufgeführt haben
        python_experience_counter += 1 
print("Das durchschnittliche Semester aller Teilnehmenden ist:", sum_semester/len(python_course))
print("Es haben", python_experience_counter, "Personen bereits Erfahrung mit Python.")

{'Name': 'Robin', 'Studiengang': 'Künstliche Intelligenz', 'Semester': '3', 'Programmiererfahrung': {'Python'}}
{'Name': 'Christina', 'Studiengang': 'Cyber Security', 'Semester': '1', 'Programmiererfahrung': {'C/C++'}}
{'Name': 'Dominik', 'Studiengang': 'Robotik', 'Semester': '2', 'Programmiererfahrung': {'SPS'}}
{'Name': 'Yvonne', 'Studiengang': 'Elektrotechnik', 'Semester': '2', 'Programmiererfahrung': {'SPS', 'C/C++'}}
{'Name': 'Billel', 'Studiengang': 'Cyber Security', 'Semester': '1', 'Programmiererfahrung': {'keine'}}
{'Name': 'Paul', 'Studiengang': 'Cyber Security', 'Semester': '2', 'Programmiererfahrung': {'Fortran', 'C/C++'}}
{'Name': 'Christian', 'Studiengang': 'Elektrotechnik', 'Semester': '1', 'Programmiererfahrung': {'Java', 'SPS'}}
{'Name': 'Rahel', 'Studiengang': 'Cyber Security', 'Semester': '2', 'Programmiererfahrung': {'Java'}}
{'Name': 'Tomi', 'Studiengang': 'FinTech', 'Semester': '1', 'Programmiererfahrung': {'Python'}}
{'Name': 'Marie', 'Studiengang': 'Künstliche I

## Aufgabe C
* Erstelle eine Liste der Namen aller Erstsemester unter Verwendung einer List Comprehension und gib diese aus

In [26]:
first_semester = [person[keys[0]] for person in python_course if int(person[keys[2]]) == 1] #Die Semesterzahl ist als Zeichenkette gespeichert und wird in einen int gecastet
print("Die Namen der Studierenden im ersten Semester lauten:", first_semester)

Die Namen der Studierenden im ersten Semester lauten: ['Christina', 'Billel', 'Christian', 'Tomi']
